In [1]:
import pandas as pd

In [2]:
# Read list exported from TradingView

fn = 'SP500_old.txt'
# fn = 'SP500.txt'
df1 = pd.read_csv(fn, sep=':', lineterminator=',', names=['Exchange', 'Ticker'])
df1['Exchange'].value_counts()

Exchange
NYSE      346
NASDAQ    149
AMEX        1
Name: count, dtype: int64

In [3]:
# Read list downloaded from State Street.
# https://www.ssga.com/us/en/intermediary/etfs/funds/spdr-sp-500-etf-trust-spy

fn = 'holdings-daily-us-en-spy.xlsx'

# Read data skipping preamble
df2 = pd.read_excel(fn, skiprows=4)

# Drop Dollar row (lacks a symbol) and postamble
r = df2[df2.isnull().all(axis=1) == True].index.tolist()[0]
df2 = df2.loc[0:r-1]
df2 = df2[df2['Name'] != 'US DOLLAR']

In [4]:
removed_tickers = df1[~df1['Ticker'].isin(df2['Ticker'])]['Ticker']
added_tickers = df2[~df2['Ticker'].isin(df1['Ticker'])]['Ticker']

In [5]:
removed_tickers

65      DXC
67      OGN
72     EMBC
88      AAP
96      NWL
158    TWTR
276     VNT
416    DISH
465     LNC
482    FBIN
494    LUMN
495     VNO
Name: Ticker, dtype: object

In [6]:
added_tickers

32      LIN
93     PANW
101      BX
103      FI
136    ABNB
137     ROP
147    LULU
221    KVUE
232     COR
263    FICO
268    GEHC
349      EG
360     COO
362    VLTO
367      BG
369    AXON
370    HUBB
411    PODD
434    RVTY
Name: Ticker, dtype: object